# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../output_data/Tina_cities.csv")
weather_data


,City ID,City,Lat,Long,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,0,saint-philippe,-21.36,55.77,73.40,64,0,5.82,RE,1603836949
1,1,beringovskiy,63.05,179.32,27.19,92,85,11.88,RU,1603836803
2,2,bredasdorp,-34.53,20.04,64.40,77,100,4.70,ZA,1603836949
3,3,ahuimanu,21.44,-157.84,80.60,74,90,4.88,US,1603836949
4,4,saint george,37.10,-113.58,61.00,9,1,3.36,US,1603836788
...,...,...,...,...,...,...,...,...,...,...
578,578,faya,18.39,42.45,60.80,19,0,4.70,SA,1603837016
579,579,axim,4.87,-2.24,78.46,86,100,8.08,GH,1603836900
580,580,mackay,-21.15,149.20,78.01,83,40,4.70,AU,1603836927
581,581,port lincoln,-34.73,135.87,58.93,73,0,6.31,AU,1603837016


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Long"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
narrowed_city_df = weather_data.loc[(weather_data["Windspeed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

narrowed_city_df

,City ID,City,Lat,Long,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,0,saint-philippe,-21.36,55.77,73.40,64,0,5.82,RE,1603836949
73,73,diu,20.71,70.98,77.74,53,0,5.79,IN,1603836957
108,108,grand gaube,-20.01,57.66,70.00,79,0,3.00,MU,1603836960
114,114,bell ville,-32.63,-62.69,75.00,48,0,5.99,AR,1603836961
115,115,vallenar,-28.57,-70.76,70.81,41,0,6.20,CL,1603836961
182,182,nioro,13.35,-15.75,78.19,48,0,4.52,GM,1603836968
192,192,tessalit,20.20,1.01,79.52,17,0,7.90,ML,1603836969
202,202,bara,10.37,10.73,75.60,38,0,1.12,NG,1603836970
295,295,saint-joseph,-21.37,55.62,73.40,64,0,5.82,RE,1603836981
341,341,abnub,27.27,31.15,71.60,64,0,6.93,EG,1603836986


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
params = {
        "radius": 5000,
        "types" : "lodging",
        "key": g_key
    }

for index, row in narrowed_city_df.iterrows():
    lat = row['Lat']
    lng = row['Long']

    params['location'] = f'{lat},{lng}'
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        narrowed_city_df.loc[index, "Hotel Name"] = jsn['results'][0]['name']
    except (KeyError, IndexError):
        print(f'Missing result, skipping..')
        
narrowed_city_df.head()

Missing result, skipping..
Missing result, skipping..
Missing result, skipping..


,City ID,City,Lat,Long,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date,Hotel Name
0,0,saint-philippe,-21.36,55.77,73.40,64,0,5.82,RE,1603836949,"Chambres d'hôte ""La Trinité"""
73,73,diu,20.71,70.98,77.74,53,0,5.79,IN,1603836957,Hotel Relax Inn
108,108,grand gaube,-20.01,57.66,70.00,79,0,3.00,MU,1603836960,Veranda Paul et Virginie Hotel & Spa
114,114,bell ville,-32.63,-62.69,75.00,48,0,5.99,AR,1603836961,Hotel Central
115,115,vallenar,-28.57,-70.76,70.81,41,0,6.20,CL,1603836961,Humacao Bed & Breakfast


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Lat", "Long"]]

In [14]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))